# Wedding Seat Optimization using Simulated Annealing

What I do when my best friends are getting married and I'm getting antsy.

In [1]:
import matplotlib.pyplot as plt
plt.style.use('ggplot')
%matplotlib inline
%config InlineBackend.figure_format = 'svg'

import pandas as pd
import numpy as np
import networkx as nx
import string

In [2]:
guest_list = list(string.ascii_uppercase)[:10]
relationships_edges = {
    ('A', 'B'): -50,
    ('C', 'D'): -50,
    ('A', 'D'): 50,
    ('E', 'F'): 25,
    ('F', 'G'): -50,
    ('H', 'I'): -50,
}
table_size = 5
table_count = len(guest_list) // table_size

In [3]:
print(guest_list)

['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J']


In [4]:
temp_graph = nx.Graph()
for k, v in relationships_edges.items():
    temp_graph.add_edge(k[0], k[1], weight=v)
relationships_mat = nx.to_numpy_matrix(temp_graph.to_undirected(), nodelist=guest_list)

print(relationships_mat)

[[  0. -50.   0.  50.   0.   0.   0.   0.   0.   0.]
 [-50.   0.   0.   0.   0.   0.   0.   0.   0.   0.]
 [  0.   0.   0. -50.   0.   0.   0.   0.   0.   0.]
 [ 50.   0. -50.   0.   0.   0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.  25.   0.   0.   0.   0.]
 [  0.   0.   0.   0.  25.   0. -50.   0.   0.   0.]
 [  0.   0.   0.   0.   0. -50.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.   0.   0.   0. -50.   0.]
 [  0.   0.   0.   0.   0.   0.   0. -50.   0.   0.]
 [  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.]]


In [5]:
table_0_seats = np.matrix([[1, 0, 1, 0, 0, 1, 1, 1, 0, 0]])
print(table_0_seats * relationships_mat * table_0_seats.T)

[[-100.]]


In [6]:
table_seats_a = np.matrix([
    [1, 0, 1, 0, 0, 1, 1, 1, 0, 0],
    [0, 1, 0, 1, 1, 0, 0, 0, 1, 1]
])
table_costs_a = table_seats_a * relationships_mat * table_seats_a.T
print(table_costs_a)
print(np.trace(table_costs_a))

[[-100.  -75.]
 [ -75.    0.]]
-100.0


In [7]:
table_seats_b = np.matrix([
    [1, 1, 1, 0, 1, 0, 0, 0, 0, 1],
    [0, 0, 0, 1, 0, 1, 1, 1, 1, 0]
])
table_costs_b = table_seats_b * relationships_mat * table_seats_b.T
print(table_costs_b)
print(np.trace(table_costs_b))

[[-100.   25.]
 [  25. -200.]]
-300.0


In [8]:
def reshape_to_table_seats(x):
    table_seats = x.reshape(table_count, len(guest_list))
    return table_seats

def cost(x):
    table_seats = reshape_to_table_seats(x)
    table_costs = table_seats * relationships_mat * table_seats.T
    table_cost = np.trace(table_costs)
    return table_cost

def take_step(x):
    table_seats = reshape_to_table_seats(np.matrix(x, copy=True))
    # randomly swap two guests
    table_from, table_to = np.random.choice(table_count, 2, replace=False)
    
    table_from_guests = np.where(table_seats[table_from] == 1)[1]
    table_to_guests = np.where(table_seats[table_to] == 1)[1]
    
    table_from_guest = np.random.choice(table_from_guests)
    table_to_guest = np.random.choice(table_to_guests)
    
    table_seats[table_from, table_from_guest] = 0
    table_seats[table_from, table_to_guest] = 1
    table_seats[table_to, table_to_guest] = 0
    table_seats[table_to, table_from_guest] = 1
    return table_seats

def prob_accept(cost_old, cost_new, temp):
    a = np.exp((cost_old - cost_new) / temp)
    return a

In [9]:
def anneal(pos_current):
    cost_old = cost(pos_current)
    temp = 1.0
    temp_min = 0.00001
    alpha = 0.9
    n_iter = 100
    
    while temp > temp_min:
        for i in range(0, n_iter):
            pos_new = take_step(pos_current)
            cost_new = cost(pos_new)
            p_accept = prob_accept(cost_old, cost_new, temp)
            # print('cost_new %.1f, cost_old %.1f, temp %.1f, p_accept %.5f' % (cost_new, cost_old, temp, p_accept))
            if p_accept > np.random.random():
                pos_current = pos_new
                cost_old = cost_new
        temp *= alpha
    return pos_current, cost_old

In [10]:
result = anneal(table_seats_b)

In [11]:
print(result[0])
print(cost(result[0]))

[[1 1 0 0 1 0 0 1 1 0]
 [0 0 1 1 0 1 1 0 0 1]]
-400.0
